# Tacit Collusion and Dynamic Competition


In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from scipy.optimize import minimize
from scipy.optimize import fsolve
import scipy

/Users/mikkeldollerup/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Model

In [2]:
def R(pi, pj): 
    if pi < pj: 
        return 1.0
    elif pi == pj: 
        return 0.5
    else: 
        return 0.0

def s(p1, p2, alpha): 
    u0 = 0.0 
    u1 = 1.0 - p1 + alpha * R(p1, p2)
    u2 = 1.0 - p2 + alpha * R(p2, p1)

#     u = np.array([u0, u1, u2]) 

#     # max-rescale utilities: probabilities are left unchanged but we get rid of numerical 
#     # underflow errors (possibly at the cost of more underflow errors, which are less problematic)
#     u -= np.max(u, keepdims=True) 

    s1 = np.exp(u1) / (np.exp(u0) + np.exp(u1) + np.exp(u2))
    s2 = np.exp(u2) / (np.exp(u0) + np.exp(u1) + np.exp(u2))
    return s1, s2

def profit1(p1, p2, alpha=1.0): 
    s1, s2 = s(p1, p2, alpha)
    return s1 * p1

def profit2(p1, p2, alpha=1.0): 
    s1, s2 = s(p1, p2, alpha)
    return s2 * p2


# Opgave 1

In [3]:
from scipy.optimize import minimize

def BR1(p2):
    f = lambda p1: -pi1(p1, p2)
    res = optimize.minimize(f, [0], bounds=[(0, None)])
    p1 = res.x[0]
    return p1

def BR2(p1):
    f = lambda p2: -pi2(p1, p2)
    res = optimize.minimize(f, [0], bounds=[(0, None)])
    p2 = res.x[0]
    return p2

In [4]:
# Initialization
p2 = 1
p1_prev = np.inf
p2_prev = np.inf
iteration = 1

while True:
    f_ = lambda p: -profit1(p1=p, p2=p2, alpha=0)  # anonymous function of one variable
    res = minimize(f_, x0=p2)
    p1 = res['x'][0]

    f_ = lambda p: -profit2(p1=p1, p2=p, alpha=0)
    res = minimize(f_, x0=p2)
    p2 = res['x'][0]

    # Check if the change is smaller than 1e-8 in both p1 and p2
    if abs(p1 - p1_prev) < 1e-8 and abs(p2 - p2_prev) < 1e-8:
        break

    p1_prev = p1
    p2_prev = p2
    #print(f'Iteration {iteration}: BR1(p2={p2:6.8f}) = {p1:6.8f},   BR2(p1={p1:6.8f}) = {p2:6.8f}')
    iteration += 1

print(f'\nBR1(p2={p2:6.8f}) = {p1:6.8f} and {iteration} iterations')
print(f'BR2(p1={p1:6.8f}) = {p2:6.8f} and {iteration} iterations')


BR1(p2=1.40105555) = 1.40105555 and 5 iterations
BR2(p1=1.40105555) = 1.40105555 and 5 iterations


In [5]:
from scipy.optimize import fsolve
from scipy.optimize import minimize

# opgave 2

In [6]:
from scipy.optimize import minimize

def cartel_profit(prices):
    p = prices[0]
    return -(profit1(p, p) + profit2(p, p))


def find_cartel_equilibrium():
    initial_price = [0] 
    result = minimize(cartel_profit, initial_price)
    cartel_prices = result.x
    return cartel_prices

cartel_prices = find_cartel_equilibrium()
print(f"Ligevægtspris hvis firmaerne indgår kartel: p1 = {cartel_prices[0]:.2f}, p2={cartel_prices[0]:.2f}")


Ligevægtspris hvis firmaerne indgår kartel: p1 = 2.10, p2=2.10


In [9]:
def cartel_p1(p, alpha=0): 
    s1, s2 = s(p, p, alpha)
    return s1 * p

def cartel_p2(p, alpha=0): 
    s1, s2 = s(p, p, alpha)
    return s2 * p

def objective(p, sign=1.0):
    return sign * (cartel_p1(p) + cartel_p2(p))
from scipy.optimize import minimize

p=1
result = minimize(objective, [p], args=(-1.0,))
optimal_prices = result.x

print("Optimal pris ved karteldannelse:")
print(f"p = {optimal_prices[0]:.3f}")


Optimal pris ved karteldannelse:
p = 1.853


# opgave 4 
Følgende trigger-strategi: Spil p_bar(0.71) og vis anden person afviger spil ligevægt p_star(0.51)

In [46]:
p_star = p1 
p_bar = optimal_prices[0]
print(p_star.round(5), p_bar.round(5))
print(type(p_star), type(p_bar))

1.44795 1.85259
<class 'numpy.float64'> <class 'numpy.float64'>


In [55]:
pi_cartel = profit1(p_bar,p_bar, alpha = 0)
pi_devi = profit1(p_star,p_star, alpha = 0)

f_ = lambda p: -profit1(p1=p, p2=p_bar,alpha = 0)
res = minimize(f_, x0 = p2)
p1 = res['x'][0]

pi_hat = res['x'][0]

pi_h = profit1(p_hat,p_bar,alpha=0)


print(f"profit ved kartel = {pi_cartel:.3f}\nprofit ved at afvige = {pi_devi:.3f}\npi_h = {pi_h:.3f}")




profit ved kartel = 0.426
profit ved at afvige = 0.406
pi_h = 0.448


In [48]:
delta = (pi_h - pi_cartel)/(pi_h - pi_devi)
print(f"delta >= {delta:.3f}")

delta >= 0.518


In [54]:
from scipy.optimize import fsolve

def returnR(r,delta):
    return 1/ (1+r) - delta
res = fsolve(returnR,x0=0,args=(delta,))

print(f"yearly return = {res[0]:.4f}")

yearly return = 0.9307


# Opgave 5